In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import json
from datetime import datetime
from pathlib import Path

try:
    import boto3
    import sagemaker
    from sagemaker import get_execution_role
    from sagemaker.sklearn import SKLearn
    print("✅ AWS libraries imported successfully")
except ImportError:
    print("⚠️ AWS SDK not installed. Installing...")
    !pip install boto3 sagemaker --quiet
    import boto3
    import sagemaker
    from sagemaker import get_execution_role
    from sagemaker.sklearn import SKLearn
    print("✅ AWS libraries installed and imported")

print(f"SageMaker version: {sagemaker.__version__}")
print(f"Notebook run: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/hamidiqbal/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/hamidiqbal/Library/Application Support/sagemaker/config.yaml
✅ AWS libraries imported successfully
SageMaker version: 2.254.1
Notebook run: 2025-11-23 17:09:59
✅ AWS libraries imported successfully
SageMaker version: 2.254.1
Notebook run: 2025-11-23 17:09:59


## 1. AWS SageMaker Setup

Configure SageMaker session and S3 bucket.

In [2]:
# Initialize SageMaker session
sess = sagemaker.Session()
role = get_execution_role()

print(f"✅ SageMaker session initialized")
print(f"Role: {role}")

# Get default S3 bucket
bucket = sess.default_bucket()
prefix = 'electricity-demand-prediction'

print(f"\nS3 bucket: {bucket}")
print(f"S3 prefix: {prefix}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # Initialize SageMaker session                                                              │
│ ❱  2 sess = sagemaker.Session()                                                                  │
│    3 role = get_execution_role()                                                                 │
│    4                                                                                             │
│    5 print(f"✅ SageMaker session initialized")                                                  │
│                                                                                                  │
│ /Users/hamidiqbal/Documents/ThomasMore/ML/cloud/project/cloud_ai_project/.venv/lib/python3.12/si │
│ te-packages/sagemaker/session.py:265 in __init__                                                 │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ /Users/hamidiqbal/Documents/ThomasMore/ML/cloud/project/cloud_ai_project/.venv/lib/python3.12/si │
│ te-packages/sagemaker/session.py:293 in _initialize                                              │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

## 2. Load and Prepare Data

Load the prepared data and prepare for upload to S3.

In [ ]:
# Load data directly from S3
s3_path = "s3://ml-cloud-project-dataset2/elec_cleaned_full.csv"

print(f"Loading data from: {s3_path}")
df = pd.read_csv(s3_path)

print(f"✅ Data loaded: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Feature engineering
# Handle datetime column
datetime_col = None
for col in df.columns:
    if 'date' in col.lower():
        datetime_col = col
        break

if datetime_col:
    df['datetime'] = pd.to_datetime(df[datetime_col], errors='coerce')
else:
    # If no date column, create from year/month/day/hour/minute
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']], errors='coerce')

# Drop rows with missing datetime or demand_value
df = df.dropna(subset=['datetime', 'demand_value'])

# Extract temporal features
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['quarter'] = df['datetime'].dt.quarter
df['week_of_year'] = df['datetime'].dt.isocalendar().week
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

# Cyclical encoding for temporal features
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

print("✅ Features extracted")
print(f"Final shape: {df.shape}")
print(f"Year range: {df['year'].min()} - {df['year'].max()}")

In [ ]:
# Prepare training data (features + target)
feature_cols = [
    'year', 'month', 'day', 'hour', 'day_of_week',
    'quarter', 'week_of_year', 'is_weekend',
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
    'day_of_week_sin', 'day_of_week_cos'
]

# Create training dataset (target as first column for SageMaker)
train_data = df[['demand_value'] + feature_cols].copy()

# Split into train/test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Training set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

## 3. Upload Data to S3

In [ ]:
# Save data locally as CSV (SageMaker format: target as first column, no header)
train_file = 'train_electricity_aws.csv'
test_file = 'test_electricity_aws.csv'

train_df.to_csv(train_file, index=False, header=False, na_rep='NaN')
test_df.to_csv(test_file, index=False, header=False, na_rep='NaN')

print(f"✅ Training data saved: {train_file}")
print(f"✅ Test data saved: {test_file}")

# Upload to S3
train_s3_path = sess.upload_data(train_file, bucket=bucket, key_prefix=f'{prefix}/train')
test_s3_path = sess.upload_data(test_file, bucket=bucket, key_prefix=f'{prefix}/test')

print(f"\n✅ Training data uploaded to: {train_s3_path}")
print(f"✅ Test data uploaded to: {test_s3_path}")

## 4. Create Training Script

Define the training script that will run on SageMaker.

In [ ]:
# Create training script directory and save script
import os

script_dir = 'sagemaker_scripts'
os.makedirs(script_dir, exist_ok=True)

training_script = os.path.join(script_dir, 'train_electricity.py')

script_content = '''import argparse
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import json

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    # Hyperparameters
    parser.add_argument('--n-estimators', type=int, default=100)
    parser.add_argument('--max-depth', type=int, default=20)
    parser.add_argument('--min-samples-split', type=int, default=10)
    parser.add_argument('--min-samples-leaf', type=int, default=5)
    
    # SageMaker specific arguments
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    
    args = parser.parse_args()
    
    print('Loading training data...')
    train_data = pd.read_csv(os.path.join(args.train, 'train_electricity_aws.csv'), header=None)
    test_data = pd.read_csv(os.path.join(args.test, 'test_electricity_aws.csv'), header=None)
    
    X_train = train_data.iloc[:, 1:].values
    y_train = train_data.iloc[:, 0].values
    X_test = test_data.iloc[:, 1:].values
    y_test = test_data.iloc[:, 0].values
    
    print(f'Training set: {X_train.shape}')
    print(f'Test set: {X_test.shape}')
    
    print('Training Random Forest model...')
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        max_depth=args.max_depth,
        min_samples_split=args.min_samples_split,
        min_samples_leaf=args.min_samples_leaf,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    
    model.fit(X_train, y_train)
    
    print('Evaluating model...')
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    metrics = {
        'mae': float(mae),
        'rmse': float(rmse),
        'r2': float(r2)
    }
    
    print(f'Test Metrics:')
    print(f'  MAE: {mae:.2f}')
    print(f'  R²: {r2:.4f}')
    
    # Print RMSE for SageMaker metric capture (must match regex pattern exactly)
    print(f'RMSE: {rmse:.2f}')
    
    # Save model
    model_path = os.path.join(args.model_dir, 'model.joblib')
    joblib.dump(model, model_path)
    print(f'Model saved to {model_path}')
    
    # Save metrics
    metrics_path = os.path.join(args.model_dir, 'metrics.json')
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f)
    print(f'Metrics saved to {metrics_path}')
'''

with open(training_script, 'w') as f:
    f.write(script_content)

print(f"✅ Training script created: {training_script}")

## 5. Train Model on SageMaker

In [ ]:
# Configure SageMaker estimator
from sagemaker.sklearn import SKLearn

sklearn_estimator = SKLearn(
    entry_point='train_electricity.py',
    source_dir=script_dir,
    role=role,
    instance_type='ml.m5.xlarge',
    framework_version='1.2-1',
    py_version='py3',
    hyperparameters={
        'n-estimators': 150,
        'max-depth': 20,
        'min-samples-split': 10,
        'min-samples-leaf': 5
    },
    output_path=f's3://{bucket}/{prefix}/output',
    base_job_name='electricity-demand-rf'
)

print("✅ SageMaker estimator configured")
print(f"Instance type: ml.m5.xlarge")
print(f"Output path: s3://{bucket}/{prefix}/output")

In [ ]:
# Train the model
print("🚀 Starting SageMaker training job...\n")
print("This will:")
print("  1. Upload training script to S3")
print("  2. Launch ml.m5.xlarge instance")
print("  3. Train Random Forest model")
print("  4. Save model artifacts to S3")
print("\nEstimated time: 5-10 minutes")
print("You can monitor progress in the SageMaker console\n")

In [ ]:
sklearn_estimator.fit({
    'train': train_s3_path,
    'test': test_s3_path
})

print("\n✅ Training complete!")

## 6. Get Training Results

In [ ]:
# Get training job details
training_info = sklearn_estimator.latest_training_job.describe()

print("\n" + "="*60)
print("SAGEMAKER TRAINING JOB COMPLETED")
print("="*60)
print(f"Job name: {training_info['TrainingJobName']}")
print(f"Status: {training_info['TrainingJobStatus']}")
print(f"Instance type: {training_info['ResourceConfig']['InstanceType']}")
print(f"Training time: {training_info['TrainingTimeInSeconds']} seconds")
print(f"Billable time: {training_info['BillableTimeInSeconds']} seconds")
print("="*60)

In [ ]:
# Download trained model
model_data_path = sklearn_estimator.model_data
print(f"Model artifacts location: {model_data_path}")

# Download model locally
import tarfile

s3 = boto3.client('s3')
model_file = 'model_electricity_aws.tar.gz'

# Extract bucket and key from model_data_path
bucket_name = model_data_path.split('/')[2]
key = '/'.join(model_data_path.split('/')[3:])

s3.download_file(bucket_name, key, model_file)
print(f"\n✅ Model downloaded to: {model_file}")

# Extract model
os.makedirs('model_extracted', exist_ok=True)
with tarfile.open(model_file, 'r:gz') as tar:
    tar.extractall(path='model_extracted')

print("✅ Model extracted to: model_extracted/")

## 7. Model Metrics

In [ ]:
# Load and display model metrics
metrics_file = 'model_extracted/metrics.json'

with open(metrics_file, 'r') as f:
    model_metrics = json.load(f)

print("\n" + "="*60)
print("MODEL TEST METRICS")
print("="*60)
print(f"MAE:  {model_metrics['mae']:,.2f} MW")
print(f"RMSE: {model_metrics['rmse']:,.2f} MW")
print(f"R²:   {model_metrics['r2']:.4f}")
print("="*60)